In [1]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    GenerationConfig
)

c:\Users\Paarth Tandon\Desktop\repos\TrashGPT\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin c:\Users\Paarth Tandon\Desktop\repos\TrashGPT\.venv\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [2]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', use_fast=True)

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          'meta-llama/Llama-2-7b-hf', quantization_config=bnb_config, device_map={"": 0}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:47<00:00, 23.70s/it]


In [3]:
peft_model = PeftModel.from_pretrained(model, "../checkpoints/checkpoint-2000")
peft_model = peft_model.to(torch.device('cuda'))

In [7]:
prompt = ''
# prompt = '-----\n\n'
# prompt = '-----\n\n00000.000 <'
# prompt = '-----\n\n00000.000 <connor>: so what were you boys up to this week'
# prompt = '-----\n\n00000.000 <connor>: what is better, bone in or boneless chicken'

inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs["input_ids"].cuda()
generation_output = peft_model.generate(
    input_ids=input_ids,
    generation_config=GenerationConfig(
        do_sample=True, temperature=1.001, top_p=0.95, top_k=50, repetition_penalty=1.001
    ),
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=512
)

for seq in generation_output.sequences:
    output = tokenizer.decode(seq)
    print(output)

<s>  it's a
00614.800 <garnt>: very weird experience because you're just
00616.079 <garnt>: not you're just not like your normal
00618.000 <garnt>: self or whatever and it's like it's like
00620.000 <garnt>: it's like it's like a very weird
00622.000 <joey>: experience going from normal to just
00623.760 <garnt>: just a complete stranger because you can
00625.360 <garnt>: only talk about in certain ways yeah
00626.799 <garnt>: yeah because like you know because of
00628.880 <garnt>: like my upbringing like the thing that
00630.960 <garnt>:'s a weird feeling for me is the fact
00632.559 <joey>: that when when a girl does do something
00634.800 <joey>: weird and like i have a joke i know
00636.720 <joey>: that's a joke i don't know a girl and
00638.800 <joey>: they do something weird and i say a
00640.320 <joey>: joke i'm worried they don't think it's a
00642.239 <joey>: joke
00644.159 <joey>: because i'm afraid like oh yeah she's
00646.800 <garnt>: actually taking that seriously oh are
